In [12]:
import torch
import pytorch_lightning as pl
import importlib
import numpy as np
import wandb
# from triplet_vade import TripletVaDE
import triplet_vade
importlib.reload(triplet_vade)
from autoencoder import SimpleAutoencoder, VaDE, ClusteringEvaluationCallback, cluster_acc, kl_distance

In [188]:
dl = triplets_model.train_dataloader()
bx, batch = next(iter(dl))

In [27]:
def kl_dist(s1, s2):
    return kl_distance(triplets_model.model.encode(batch[s1]), triplets_model.model.encode(batch[s2]))


((kl_dist('anchor', 'positive') < kl_dist('anchor', 'negative')).float().mean())



tensor(0.9336)

In [194]:
logits1 = classification_logits(triplets_model, batch['anchor'])
logits2 = classification_logits(triplets_model, batch['positive'])
logits3 = classification_logits(triplets_model, batch['negative'])


(logits1.argmax(1) == logits2.argmax(1)).float().mean()

tensor(0.9102)

In [202]:
js_dist(logits1.softmax(1), logits2.softmax(1))

tensor([3.4307e-03, 1.7740e-06, 4.6988e-05, 3.3933e-05, 6.9089e-01, 1.4499e-04,
        1.0572e-02, 1.3569e-05, 2.1554e-06, 7.3829e-06, 3.6992e-04, 8.9924e-05,
        5.6061e-04, 6.8881e-01, 3.8643e-01, 6.7821e-08, 6.5743e-01, 7.8292e-05,
        3.0658e-04, 7.2796e-04, 7.7148e-04, 4.3285e-06, 1.0464e-03, 3.0673e-06,
        4.2285e-05, 1.1209e-03, 5.6577e-03, 1.5430e-01, 6.9906e-07, 3.0518e-03,
        2.2684e-03, 6.6880e-06, 2.9676e-03, 1.2777e-04, 1.4438e-03, 3.0864e-02,
        9.1661e-02, 4.4575e-06, 2.6054e-03, 9.5361e-06, 1.9702e-05, 6.9242e-01,
        2.7807e-05, 2.9177e-05, 3.7992e-04, 6.8373e-01, 1.1303e-01, 5.4264e-02,
        2.2608e-03, 5.7824e-01, 6.9210e-01, 3.1212e-07, 7.0604e-06, 9.8527e-06,
        6.1479e-03, 6.9249e-01, 1.8267e-03, 1.6830e-04, 1.3631e-04, 6.8910e-05,
        9.5341e-02, 6.1158e-05, 1.9618e-05, 6.9312e-01, 1.3220e-05, 1.0437e-04,
        2.2238e-03, 7.4122e-05, 4.6710e-04, 5.5599e-05, 1.5851e-05, 2.7404e-04,
        1.4334e-05, 3.7041e-04, 8.6254e-

In [211]:
def classification_logits(self, bx):
        self = self.model
        z_dist = self.encode(bx)
        z = z_dist.rsample()
        log_p_z_c = self.component_distribution.log_prob(z)
        log_q_c_z = torch.log_softmax(log_p_z_c + self.mixture_logits, dim=-1)  # dims: (bs, k)
        return log_q_c_z


def kl_div_(logits1, logits2):
    return (logits1.softmax(dim=1) * (logits1 - logits2)).sum(dim=-1)

def kl_div(p, q):
    log_diff = p.log() - q.log()
    # import pdb; pdb.set_trace()
    # log_diff[p==0] = 0
    return (p * log_diff).sum(dim=1)

def js_dist(p, q, eps=1e-8):
    p, q = (p + eps), (q + eps)
    p, q = p / p.sum(dim=1, keepdims=True), q / q.sum(dim=1, keepdims=True)
    m = (p + q) / 2
    return (kl_div(p, m) + kl_div(q, m)) / 2


def js_dist_(logits1, logits2):
    M_probs = (logits1.softmax(dim=1) + logits2.softmax(dim=1)) / 2
    return (kl_div(logits1, M_probs.log()) + kl_div(logits2, M_probs.log())) // 2

def logits_kl_js_dist(s1, s2):
    logits1 = classification_logits(triplets_model, batch[s1])
    logits2 = classification_logits(triplets_model, batch[s2])
    return js_dist(logits1.softmax(1), logits2.softmax(1))

# logits1 = classification_logits(triplets_model, batch['anchor'])
# logits2 = classification_logits(triplets_model, batch['positive'])
M_probs = ((logits1.softmax(dim=-1) + logits2.softmax(dim=-1)) / 2)
# kl_div(logits1.softmax(dim=1), M_probs)
log_diff = logits1 - M_probs.log()
js_dist(logits1.softmax(1), logits2.softmax(1))
(logits_kl_js_dist('anchor', 'positive') + 0.1 < logits_kl_js_dist('anchor', 'negative')).float().mean()

tensor(0.9102)

In [142]:
torch.where(M_probs.log().isinf()), torch.where(logits2.softmax(dim=-1)==0)

((tensor([  0,   5,   7,   9,  10,  14,  15,  16,  17,  17,  18,  19,  19,  19,
           23,  24,  25,  25,  31,  33,  35,  35,  37,  38,  41,  42,  43,  45,
           47,  48,  50,  55,  56,  58,  59,  62,  63,  64,  65,  68,  68,  71,
           74,  75,  76,  80,  82,  83,  84,  87,  92,  93,  97,  97,  99, 105,
          105, 106, 107, 108, 109, 111, 117, 121, 124, 128, 130, 131, 134, 135,
          136, 137, 139, 139, 140, 142, 144, 148, 149, 150, 151, 153, 160, 161,
          162, 165, 166, 167, 168, 169, 171, 172, 174, 176, 177, 180, 181, 187,
          190, 190, 191, 192, 193, 196, 197, 198, 201, 202, 204, 205, 206, 212,
          213, 214, 215, 217, 218, 222, 224, 227, 228, 234, 236, 240, 241, 244,
          246, 247, 248, 249, 250, 251, 252, 253, 255]),
  tensor([4, 4, 4, 4, 7, 4, 4, 4, 0, 7, 4, 0, 4, 7, 4, 4, 0, 7, 4, 4, 0, 7, 4, 4,
          4, 4, 4, 4, 4, 4, 4, 7, 4, 4, 7, 4, 7, 4, 4, 0, 4, 4, 4, 4, 7, 4, 4, 7,
          4, 4, 4, 4, 0, 7, 4, 0, 4, 4, 4, 4, 4, 7, 4, 4, 4

In [3]:
pretriained_model = 'pretrained_models/radiant-surf-28/autoencoder-epoch=55-loss=0.011.ckpt'
hparams = {'lr': 2e-3, 'lr_gmm': 2e-3, 'batch_size': 2**8, 'pretrain_epochs': 50, 'triplet_loss_margin': 0.5, 'triplet_loss_alpha': 0.0, 'warmup_epochs':10, 'triplet_loss_alpha_kl': 1., 'n_samples_for_triplets': None, 'data_size': None, 'pretrained_model_file': pretriained_model}
for i in range(1):
    triplets_model = triplet_vade.TripletVaDE(n_neurons=[784, 512, 512, 2048, 10], **hparams)
    # wandb.init()
    logger =  pl.loggers.wandb.WandbLogger(project='VaDE Triplets', group='triplet_loss_limited_data')

    callbacks = [ClusteringEvaluationCallback(on_start=True), 
                 ClusteringEvaluationCallback(on_start=True, ds_type='valid'),
                 pl.callbacks.LearningRateMonitor()]
    logger.log_hyperparams(hparams)
    trainer = pl.Trainer(gpus=1, callbacks=callbacks, logger=logger, 
                         log_every_n_steps=10, progress_bar_refresh_rate=10, max_epochs=50)
    trainer.fit(triplets_model)
    wandb.finish()
    # triplets_model.save_hyperparameters()

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: shukistern (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.22 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | VaDE | 3.5 M 
-------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

/home/shuki/anaconda3/envs/torch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Validating: 0it [00:00, ?it/s]

/home/shuki/anaconda3/envs/torch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


lr-AdamW/pg1,0.00131
lr-AdamW/pg2,0.00131
_step,11749
_runtime,685
_timestamp,1615735724
train/latent mu mean,0.02941
train/latent mu std,0.04979
train/latent dist std,0.12895
train/loss,91.53633
train/x_recon_loss,72.09731
train/kl_loss,19.43902


lr-AdamW/pg1,▁▂▃▄▅▆▇█████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆
lr-AdamW/pg2,▁▂▃▄▅▆▇█████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/latent mu mean,▅▇███▇▆▆▅▅▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▁▁▁▁▁
train/latent mu std,▅▆▇██▇▆▆▅▅▄▄▄▃▄▃▃▃▃▃▂▂▃▂▂▁▁▂▂▁▁▁▂▁▂▁▁▁▁▁
train/latent dist std,▁▄▄▄▃▅▆▆▆▅▇▇▇▇█▅▆▆▇▇▇▆▇▅▇█▆▇▇▆▆▆▆▇▆▆▆▅▆▇
train/loss,▆▃▄▃▄▅▄▃▇▆▆▂▂▇▃█▂▄▄▄▂▄▅▄▆▃▂▃▁▆▂▅▂▂▄▃▃▃▁▁
train/x_recon_loss,▄▃▃▃▃▅▃▃▇▅▆▂▂▇▃█▂▄▃▄▂▄▅▄▆▂▂▃▁▆▂▅▂▃▄▃▃▄▁▁
train/kl_loss,█▅▆▆▇▇▆▄▆▅▅▅▅▅▃▄▃▄▅▄▂▄▄▂▄▄▂▂▂▄▂▄▂▂▃▃▄▁▂▃


In [ ]:
hparams = {'lr': 2e-3, 'lr_gmm': 2e-3, 'batch_size': 2**8, 'pretrain_epochs': 50, 'triplet_loss_margin': 0.5, 'triplet_loss_alpha': 00.0, 'warmup_epochs':10, 'triplet_loss_alpha_kl': 1., 'n_samples_for_triplets': 1000}
for i in range(10):
    triplets_model = triplet_vade.TripletVaDE(n_neurons=[784, 512, 512, 2048, 10], **hparams)
    # wandb.init()
    logger =  pl.loggers.wandb.WandbLogger(project='VaDE Triplets', group='triplet_loss_limited_data')

    callbacks = [ClusteringEvaluationCallback(on_start=True), 
                 ClusteringEvaluationCallback(on_start=True, ds_type='valid'),
                 pl.callbacks.LearningRateMonitor()]
    logger.log_hyperparams(hparams)
    trainer = pl.Trainer(gpus=1, callbacks=callbacks, logger=logger, 
                         log_every_n_steps=10, progress_bar_refresh_rate=10, max_epochs=100)
    trainer.fit(triplets_model)
    wandb.finish()

In [19]:
model.load_state_dict(torch.load('trained_models/AE1.h5'))

<All keys matched successfully>

In [8]:
from sklearn import metrics

model.cuda()
true_y, predicted_y, X_encoded = model.cluster_data(method='kmeans')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

true_y, predicted_y, X_encoded = model.cluster_data(method='gmm-full')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

true_y, predicted_y, X_encoded = model.cluster_data(method='gmm-diag')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

0.7538434361452869
0.8249112629202103
0.7238449211641871


In [10]:
from sklearn.mixture import GaussianMixture
scores = []
for i in range(5):
    gmm = GaussianMixture(n_components=10, covariance_type='full', n_init=5)
    predicted_y = gmm.fit_predict(X_encoded)
    scores.append(metrics.normalized_mutual_info_score(true_y, predicted_y))
    
print(scores)
print(np.mean(scores), np.max(scores), np.std(scores))

[0.8206645765011308, 0.8503932657020613, 0.7995011494626447, 0.8239846028460401, 0.8483506486929793]
0.8285788486409713 0.8503932657020613 0.018952826999921197
